In [8]:
import requests
import json
import forecastio
from pprint import pprint
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from hawaii_airbnb import hawaii_airbnb
import datetime as dt

importing Jupyter notebook from hawaii_airbnb.ipynb


hawaii_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
hawaii_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
hawaii_airbnb.ipynb:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "outputs": [],


In [9]:
api_key = "a36c8345cc74c469a50078545e4970de"

In [10]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("Honolulu, HI")

HI_coord = []
HI_coord.append(location.latitude)
HI_coord.append(location.longitude)

HI_coord

[21.3250785, -157.846644178573]

In [11]:
forecast = forecastio.load_forecast(api_key, HI_coord[0], HI_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Rain today and next Friday, with high temperatures rising to 82°F on Thursday.
rain
80.18


In [12]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-21',
 '2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28']

In [15]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [16]:
hawaii_airbnb.dtypes

Date         object
Under $50     int64
$50-100       int64
$100-150      int64
$150-200      int64
$200-250      int64
dtype: object

In [17]:
HI_table = pd.merge(df, hawaii_airbnb, on="Date")
HI_table

,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250
0,2018-12-21,77.55,69.09,0.16,0.0043,rain,251,1102,1074,858,580
1,2018-12-22,78.38,69.40,0.18,0.0010,rain,198,984,968,710,505
2,2018-12-23,78.19,68.62,0.26,0.0013,rain,210,931,851,636,488
3,2018-12-24,78.70,69.01,0.20,0.0007,rain,190,782,633,476,336
4,2018-12-25,80.18,69.00,0.37,0.0026,rain,162,663,508,362,256
5,2018-12-26,81.02,70.23,0.13,0.0003,rain,175,615,425,307,205
6,2018-12-27,82.23,69.80,0.14,0.0001,rain,168,519,352,260,157
7,2018-12-28,80.52,65.64,0.95,0.0258,rain,153,480,315,227,110
